In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.info()

**we dont have any null value**
* if you look at the age column see max is 97 we try to convert it into 3 class 1) mid-young 2) old 3) very-old you can change the  name of class **

In [ ]:
df['age'] = pd.cut(df['age'] ,3,labels=["mid-young", "old", "very-old"])

* now see the pesent of each class we have in our dataframe 

In [ ]:
df['age'].value_counts().plot(kind='pie', shadow=True, autopct='%.0f%%')

* lets look at the persent of people has anaemia  
* you can use this plot for smoke and diabetes column jusr change  it this value df['?'] 

In [ ]:
df['anaemia'].value_counts().plot(kind='pie', shadow=True, autopct='%.0f%%')

* lets see which sex is more try to smoke men or women befor see the result guess which one?

In [ ]:
smoke_sex = df.groupby(['smoking', 'sex']).count()['age']

In [ ]:
smoke_sex.plot(kind='barh', grid=True)

* that's interesting now let's see how many dead if they will smoke 

In [ ]:
df.pivot_table(index='sex', columns='smoking', values='DEATH_EVENT', aggfunc='size')

* let's try another task that's help us to underestand how many pepole has both anaemia and diabetes

* 98 people no anaemia and diabetes great
* 72 people does not have anoemia but has diabetes 
* 76 people has anoemia and does not have diabetes
* 53 people has both anoemia and diabetes

In [ ]:
df.groupby(['anaemia', 'diabetes']).size()

In [ ]:
ana_dia_sm = df.pivot_table(index=['anaemia','diabetes','smoking'], aggfunc='size')
ana_dia_sm.plot(kind='barh', grid=True, color='brown')

* ready for clean the data and put label ? 

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
# use LabelEncoder to put label 
# use standardscaler for scale the data that important for Ridge , Svm and Sgd

In [ ]:
label_en = LabelEncoder()
standard = StandardScaler()
df['age'] = label_en.fit_transform(df['age'])
df.drop('time', axis='columns', inplace=True)

## looks great but as much as we need look at the Platelets it's varient value let's scale it 

In [ ]:
df

# Normalize The data :

In [ ]:
df['creatinine_phosphokinase'] /= 7861
df['serum_sodium'] /= 148
df['platelets'] /= 850000.0

In [ ]:
df['ejection_fraction'] /= 80 
df['serum_creatinine'] /= 9.4

# Scale The Data:

In [ ]:
df['creatinine_phosphokinase'] = standard.fit_transform(df['creatinine_phosphokinase'].values.reshape(-1, 1))
df['serum_sodium']= standard.fit_transform(df['serum_sodium'].values.reshape(-1, 1))
df['platelets'] = standard.fit_transform(df['platelets'].values.reshape(-1, 1))


# it's look like better:

In [ ]:
df

# Split The Data:

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# *important lets use GridsearchCV to find best param for our model* 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## *SUPPORT VECTOR MACHINE*

In [ ]:
from sklearn.svm import SVC

param_grid = [
  {'C': [1, 10, 50, 100, 200, 250], 'kernel': ['linear']},
  {'C': [1, 10, 50, 100, 200], 'gamma': [10 , 1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
 ]

In [ ]:
clf_svm = SVC()
grid_clf = GridSearchCV(clf_svm, param_grid)
grid_clf.fit(x_train, y_train)
grid_clf.best_params_

In [ ]:
predict_svm = grid_clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predict_svm))

**Confuction matrix**
* True Positive:

Interpretation: You predicted positive and it’s true.


* True Negative:

Interpretation: You predicted negative and it’s true.


* False Positive: (Type 1 Error)

Interpretation: You predicted positive and it’s false.


* False Negative: (Type 2 Error)

Interpretation: You predicted negative and it’s false.


In [ ]:
confusion_matrix(y_test.values, predict_svm)

# SGD Classifier:

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
clf_sgd = SGDClassifier(penalty='l2', max_iter=62)

clf_sgd.fit(x_train, y_train)

predict_sgd = clf_sgd.predict(x_test)

In [ ]:
print(classification_report(y_test, predict_sgd))

In [ ]:
confusion_matrix(y_test, predict_sgd)

# DecisionTree Classifier:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf_tree = DecisionTreeClassifier(max_depth=8,criterion='entropy')
clf_tree.fit(x_train, y_train)
predict_tree = clf_tree.predict(x_test)

In [ ]:
print(classification_report(y_test, predict_tree))

In [ ]:
confusion_matrix(y_test, predict_tree)

In [ ]:
from sklearn.decomposition import PCA

# now try to do some interesting task like PCA with save 95% of Our data
* pca = PCA(n_components = 0.95)
* pca = PCA(n_components = 2)
* x_2d = pd.DataFrame(pca.fit_transform(x))

# Random Foreset

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_random = RandomForestClassifier(random_state=200, max_depth=10)

In [ ]:
clf_random.fit(x_train, y_train)
predict_random = clf_random.predict(x_test)
print(classification_report(y_test, predict_random))
confusion_matrix(y_test, predict_random)

# GradientBoosting:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(max_depth=3)

In [ ]:
gbc.fit(x_train, y_train)

In [ ]:
predict_gbc = gbc.predict(x_test)

In [ ]:
confusion_matrix(y_test, predict_gbc)

In [ ]:
print(classification_report(y_test, predict_gbc))


# ROC Curve:

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict

In [ ]:
def plot_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1],[0,1], 'k--')

In [ ]:
y_score_tree = cross_val_predict(clf_tree, x_train, y_train)
fpr_tree, tpr_tree , threshold_tree = roc_curve(y_train, y_score_tree)

In [ ]:
y_score_random = cross_val_predict(clf_random, x_train, y_train)
fpr_random, tpr_random , threshold_random = roc_curve(y_train, y_score_random)

In [ ]:
y_score_svm = cross_val_predict(clf_svm, x_train, y_train)
fpr_svm, tpr_svm , threshold_svm = roc_curve(y_train, y_score_svm)

In [ ]:
y_score_sgd = cross_val_predict(clf_sgd, x_train, y_train)
fpr_sgd, tpr_sgd , threshold_sgd = roc_curve(y_train, y_score_sgd)

In [ ]:
y_score_gbc = cross_val_predict(gbc, x_train, y_train)
fpr_gbc, tpr_gbc , threshold_gbc = roc_curve(y_train, y_score_gbc)

In [ ]:
plot_curve(fpr_svm, tpr_svm, label='svm')
plot_curve(fpr_random, tpr_random, label='randomForest')
plot_curve(fpr_tree, tpr_tree, label='tree')
plot_curve(fpr_sgd, tpr_sgd, label='sgd')
plot_curve(fpr_gbc, tpr_gbc, label='gbc')

plt.legend()
plt.show()


In [ ]:
random_acc = accuracy_score(y_test,predict_random)
random_acc

In [ ]:
gbc_acc = accuracy_score(y_test,predict_gbc)
gbc_acc

In [ ]:
sgd_acc = accuracy_score(y_test,predict_sgd)
sgd_acc

In [ ]:
svm_acc = accuracy_score(y_test,predict_svm)
svm_acc

In [ ]:
tree_acc = accuracy_score(y_test,predict_tree)
tree_acc

In [ ]:
tree_acc = pd.Series(tree_acc)

In [ ]:
score = {'Decision_tree': tree_acc, 
         'Svm': svm_acc,
         'Sgd': sgd_acc,
          'RandomForest': random_acc,
           'GradienBoost': gbc_acc }

In [ ]:
df = pd.DataFrame(data=score)

In [ ]:
df.index.name = 'score'

In [ ]:
a = df.plot(kind='bar', figsize=(8, 5), ylabel='Accuracy')
